## Consignas del desafío 1

### 1. Vectorizar documentos

Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

---

Para resolver esta consigna lo importante es, en primer lugar, obtener un dataset con documentos variados y vectorizar los documentos usando la técnica TF-IDF. Al igual que lo mostrado en clase, se recurre al dataset de documentos [20 News Groups](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) que ya está disponible en las librerías de *Scikit Learn* y contiene 18000 documentos distintos categorizados en 20 tópicos ya separados en conjuntos de entrenamiento y validación. La carga de los datos es como se muestra a continuación:

In [8]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

La vectorización mediante TF-IDF (*Term Frequency - Inverse Document Frequency*) consiste en la generación de una lista de **vocabulario** compuesta de todas las palabras presentes en todos los documentos sobre la que se cuenta la frecuencia de aparición de cada palabra en cada documento, de modo que se puede relacionar cada documento con un vector (TF). A continuación se evalúa qué tan informativa o única es cada palabra del vocabulario de modo que las palabras que aparezcan menos en el corpus completo presenten un mayor índice (IDF). El índice IDF de una palabra $n$ es tradicionalmente calculado como:

$$
IDF(n) = log(\frac{N}{DF(n)})
$$

Donde $N$ es la cantidad total de documentos en el corpus y $DF(n)$ la cantidad de documentos que contienen al menos una aparición del término evaluado.

Finalmente, para completar la caracterización del vectorizado de cada documento se multiplica el vector TF de cada documento por el valor del índice IDF de cada palabra en el vector.

Todo este proceso puede ser realizado utilizando vectorizadores ya provistos por *Scikit Learn* como se muestra a continuación:

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test = vectorizer.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test = newsgroups_test.target

La matriz obtenida es también llamada **"Matriz documento-término"**.

Para elegir 5 documentos al azar simplementese utiliza el método `random.sample` para obtener 5 índices aleatorios sobre los que extraer los documentos.

Para observar la similitud entre cada documento elegido y el resto del corpus se puede utilizar la similitud del coseno entre el vector elegido y los demás para encontrar los vectores más cercanos entre sí. La función `cosine_similarity` permite hacer el cálculo de manera óptima; y si luego se ordenan los valores de la lista resultante se pueden obtener los 5 documentos más similares al evaluado para ver si la etiqueta asignada es la correcta:

In [7]:
import random
from sklearn.metrics.pairwise import cosine_similarity

# Select 5 random document indices
random_indices = random.sample(range(X_train.shape[0]), 5)

i = 0
for idx in random_indices:
    i += 1
    print(f"\n====== Documento N°{i} ======")

    # Calculate cosine similarity between the selected document and all others
    similarities = cosine_similarity(X_train[idx], X_train).flatten()

    # Get indices of the 5 most similar documents (excluding itself)
    most_similar = similarities.argsort()[::-1][1:6]
    
    print(f"\nÍndice del documento aleatorio: {idx}")
    print("Texto (acotado):", ' '.join(newsgroups_train.data[idx].split()[:50]))
    print("Grupo:", newsgroups_train.target_names[newsgroups_train.target[idx]])
    
    print("\n--- Documentos similares ---")
    for sim_idx in most_similar:
        print(f"\nÍndice: {sim_idx}")
        print("Texto (acotado):", ' '.join(newsgroups_train.data[sim_idx].split()[:50]))
        print("Grupo:", newsgroups_train.target_names[newsgroups_train.target[sim_idx]])
        print(f"Similitud: {similarities[sim_idx]:.2f}")


====== Documento N°1 ======

Índice del documento aleatorio: 1096
Texto (acotado): Roland HS-60 (SynthPlus 60) - 6 voice polyphonic fully programmable analog synthesizer - 61 full size keys - MIDI - Memory to store 128 patches - built in speakers - Connectors: MIDI in thru out, Input(2), Expansion Pedal, Head Phones, Tape Load & Save, Patch Shift, Pedal Hold, Output(2) Asking
Grupo: misc.forsale

--- Documentos similares ---

Índice: 9012
Texto (acotado): : I have a 486DX-33 computer with a SoundBlaster 1.0 card. I'm running : Microsoft Windows v3.1. I have the SB driver set up properly to play : normal sounds (.WAV files, etc.). I want to play midi files through the : Media Player that is included with windows. I
Grupo: comp.os.ms-windows.misc
Similitud: 0.25

Índice: 10012
Texto (acotado): Does anyone have any experience using LCIII with MIDI? Do they get along OK? I have heard that the IIvx is not suitable for MIDI, but I have not heard anyon e say anything definitive about LCIII a

En líneas generales, se observa que los valores de similitud entre documentos no son muy elevados, pero el grupo en el que se clasifican el elemento aleatorio y los que se han calculado ser más similares suelen ser el mismo. Leyendo las frases extraídas de cada documento evaluado para cada experimento uno puede observar que tienden a hablar del mismo tópico, confirmando la clasificación en los grupos etiquetados.

### **2**. Construir un modelo de clasificación por prototipos (tipo zero-shot)

Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

---

Para cumplir esta consigna simplemente se compara la similitud del coseno de cada elemento del conjunto de validación con el corpus de entrenamiento y se otorga al elemento de evaluación el mismo grupo que el del elemento más cercano encontrado (un análogo a un algoritmo KNN con $K=1$ que usa la similitud del coseno como distancia entre elementos)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

predicted_labels = []
for x_test_vec in X_test:
    sims = cosine_similarity(x_test_vec, X_train).flatten()

    # Get index of the most similar document
    most_similar_idx = np.argmax(sims)
    predicted_labels.append(newsgroups_train.target[most_similar_idx])

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"\nCerteza en la clasificación del conjunto de validación: {accuracy:.4f}")


Certeza en la clasificación del conjunto de validación: 0.5089


Este método ruidimentario es muy sensible a ruido en las clasificaciones encontradas, pero es simple y permite constituir un baseline con el que contrastar otros modelos en el práctico. El valor que se apunta a superar es el 50.89% de certeza global (u otra métrica mejor para la clasificación multigrupo como el F1-score).

### **3**. Entrenar modelos de clasificación Naïve Bayes 

Maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

---

Nuevamente, *Scikit Learn* provee modelos de clasificadores que se pueden entrenar para evaluar el desempeño. Se generan varias versiones de estos modelos, jugando con sus parámetros y comparando los resultados obtenidos, medidos de acuerdo a la puntuación F1.

Además, se hacen pruebas con modificando los siguientes parámetros del vectorizador:

* `ngram_range`: Cantidad de palabras evaluadas a la vez

* `min_df` y `min_df`: Controlan la frecuencia mínima y máxima, respectivamente, con la que debe aparecer una palabra en el corpus para ser considerada por el modelo

* `stop_words`: Eliminan términos contenidos en un diccionario con palabras de uso común que no aportan para diferenciar un documento

In [15]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Searching grid for the best alpha parameter for MultinomialNB & ComplementNB
param_grid = {
    "alpha": [0.1, 0.5, 1.0]
}

# State all models to train and evaluate
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

# Function to train and evaluate a model using GridSearchCV
def train_and_evaluate(model, model_name, X_train, y_train, X_test, y_test):
    print(f"\n--- Modelo {model_name} ---")

    # Use GridSearchCV to find the best alpha
    grid = GridSearchCV(model, param_grid, scoring="f1_macro", cv=5)
    grid.fit(X_train, y_train)
    print("Mejor alpha:", grid.best_params_)
    print("Mejor F1-macro (train):", grid.best_score_)

    # Evaluate on the test set
    model = grid.best_estimator_
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average="macro")
    print(f"F1-macro (test): {f1:.3f}")

# Training and evaluating each model
print("===== Vectorizador original =====")
for name, model in models.items():
    train_and_evaluate(model, name, X_train, y_train, X_test, y_test)

# Modifying the vectorizer with new parameters
modified_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', max_df=0.9, min_df=3, ngram_range=(1,2))
X_train_modified = modified_vectorizer.fit_transform(newsgroups_train.data)
X_test_modified = modified_vectorizer.transform(newsgroups_test.data)

# Use new encodings to train and evaluate each model again
print("===== Vectorizador modificado =====")
for name, model in models.items():
    train_and_evaluate(model, name, X_train_modified, y_train, X_test_modified, y_test)

===== Vectorizador original =====

--- Modelo MultinomialNB ---
Mejor alpha: {'alpha': 0.1}
Mejor F1-macro (train): 0.7188497427747561
F1-macro (test): 0.656

--- Modelo ComplementNB ---
Mejor alpha: {'alpha': 0.1}
Mejor F1-macro (train): 0.7656097292010164
F1-macro (test): 0.695
===== Vectorizador modificado =====

--- Modelo MultinomialNB ---
Mejor alpha: {'alpha': 0.1}
Mejor F1-macro (train): 0.7521240433003771
F1-macro (test): 0.680

--- Modelo ComplementNB ---
Mejor alpha: {'alpha': 0.5}
Mejor F1-macro (train): 0.7665934794124383
F1-macro (test): 0.703


Los resultados obtenidos se sintetizan en la siguiente tabla:

| Modelo                                  | Mejor α | F1-macro (train) | F1-macro (test) |
|-----------------------------------------|---------|------------------|-----------------|
| MultinomialNB (Vectorizador Original)   | 0.1     | 0.719            | 0.656           |
| ComplementNB (Vectorizador Original)    | 0.1     | 0.766            | 0.695           |
| MultinomialNB (Vectorizador Modificado) | 0.1     | 0.752            | 0.680           |
| ComplementNB (Vectorizador Modificado)  | 0.5     | 0.767            | 0.703           |

En cualquiera de estas versiones se obtienen mejores resultados que el clasificador zero-shot implementado inicialmente por un margen de mejora de entre el 30 y el 40%. De entre los modelos evaluados se observa mejores resultados en aquellos que usan el vectorizador modificado tanto en el conjunto de entrenamiento como en el de evaluación, destacando la obtención de un modelo con un **70%** de certeza en el conjunto de evaluación cuando las mejoras en el vectorizador se han aplicado.

### **4**. Transponer la matriz documento-término

De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

---

Para poder cumplir esto simplemente se realiza el mismo proceso que se hizo para detectar la similitud entre documentos (cada documento es representado por una fila) usando la similtud del coseno pero comparando palabras esta vez (cada palabra es representada por una columna). Esto se logra simplemente transponiendo la matriz documento-término que se obtuvo con anterioridad.

Se usará para esto la vectorización modificada que se generó en el ejercicio anterior, ya que contiene un contexto reducido y mejor tratado de las palabras en el vocabulario. Además, como se preparó la vectorización para consumir las palabras de a pares, las palabras evaluadas podrán ser devueltas también de a pares, entendiendo mejor el contexto en el que se suelen usar:

In [16]:
# Selection of 5 specific words to analyze their 5 most context-similar words
word_scope = ["computer", "devil", "guns", "conflict", "bat"]

# Transpose the modified TF-IDF matrix to get term-document matrix & calculate word-similarities
similarities = cosine_similarity(X_train_modified.T)

# List words on the vocabulary
feature_names = modified_vectorizer.get_feature_names_out()

# Find and print the 5 most similar words for each word in word_scope
for word in word_scope:
    if word in feature_names:
        idx = list(feature_names).index(word)
        sims = similarities[idx]
        top5_idx = np.argsort(sims)[::-1][1:6]  # excluye la misma palabra
        print(f"\nPalabra base: {word}")
        for i in top5_idx:
            print(f"   - {feature_names[i]} (sim={sims[i]:.3f})")
    
    else:
        print(f"La palabra '{word}' no está en el vocabulario.")


Palabra base: computer
   - computer science (sim=0.225)
   - computer graphics (sim=0.220)
   - turn computer (sim=0.207)
   - computer equipment (sim=0.176)
   - new computer (sim=0.169)

Palabra base: devil
   - honig (sim=0.336)
   - copenhagen denmark (sim=0.335)
   - copenhagen (sim=0.318)
   - diku dk (sim=0.317)
   - diku (sim=0.317)

Palabra base: guns
   - gun (sim=0.388)
   - politics guns (sim=0.329)
   - talk politics (sim=0.272)
   - machine guns (sim=0.261)
   - make guns (sim=0.245)

Palabra base: conflict
   - variety sources (sim=0.324)
   - hardware conflict (sim=0.294)
   - israeli palestinian (sim=0.277)
   - calamity (sim=0.276)
   - religious differences (sim=0.275)

Palabra base: bat
   - autoexec bat (sim=0.436)
   - autoexec (sim=0.431)
   - bat file (sim=0.305)
   - config sys (sim=0.294)
   - sys autoexec (sim=0.262)


Se observa que los términos encontrados con mayor similitud incluyen el uso de expresiones, lugares, disciplinas o acepciones de las palabras que no se consideraron originalmente